## Introduction

TEAMMATES: Akshat and Annie

The overall goal is to predict whether a payment by a company to a medical doctor or facility
was made as part of a research project or not.

### Imports

In [111]:
# data loading and manipulation
import pandas as pd
import numpy as np
import random
# from dirty_cat import TargetEncoder
from category_encoders import TargetEncoder

# scikit learn
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error, confusion_matrix, roc_auc_score, auc, average_precision_score

# unbalanced sets
from imblearn.under_sampling import RandomUnderSampler

# plotting
import matplotlib.pyplot as plt

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Load data

The positive class corresponds to the payments that were made by a company to a doctor or facility that is part of the **research project**. The negative class on the other hand are the **general payments**. 

In the original data sets, the ratio of the positive class to the negative class is 1/20, making the positive class the minority class. 

Because the data sets are so large, we will subsample from the classes in order to maintain the same ratio. Thus we take 120K data points from Class 0, and 20K data points from Class 1. 

120K from the positive class turns out to be ~20% of the data, and 2M from the negative class is ~20% from the negative class. 

In [2]:
# Import 20% data randomly
# p = 0.2
# df0 = pd.read_csv('../payments2017/d0.csv', skiprows=lambda i: i>0 and random.random() > p)
# df1 = pd.read_csv('../payments2017/d1.csv', skiprows=lambda i: i>0 and random.random() > p)

# Write sampled data for future use
# df0.to_csv('../payments2017/gen_payments_sampled.csv')
# df1.to_csv('../payments2017/res_payments_sampled.csv')

In [3]:
# Import from sampled files
df0 = pd.read_csv('../payments2017/gen_payments_sampled.csv')
df1 = pd.read_csv('../payments2017/res_payments_sampled.csv')

In [4]:
df0.shape

(2132686, 76)

In [5]:
df1.shape

(120511, 177)

## Feature Intersection

What features should be excluded because they leak the target information?

There are 75 features present in the negative class, and 176 in the positive class. Our approach to combining the data sets for both the positive and the negative classs it to take an intersection of the features. 

In [6]:
notPrs = list(set(list(df1.columns)).difference(list(df0.columns)))
featureIntersection = list(set(list(df1.columns)).difference(notPrs))
print("There are {} features present in the intersection of the two dataframes.".format(len(featureIntersection) - 1))

df1 = df1[featureIntersection]
df0 = df0[featureIntersection]

There are 64 features present in the intersection of the two dataframes.


Before we concatenate the two data sets, we add an indicator variable to each one specifying which class the data belongs to. We call this feature **target**, which is equal to 1 for the positive class and 0 for the negative class.

In [7]:
df1['Target'] = 1
df0['Target'] = 0

df = pd.concat([df1, df0], axis=0)
df.shape

(2253197, 66)

## NA Columns

In [8]:
NAs = df.isna().mean().sort_values(ascending=False)

In [9]:
NAs

Recipient_Province                                                  0.999945
Recipient_Postal_Code                                               0.999931
Physician_License_State_code5                                       0.999826
Physician_License_State_code4                                       0.999209
Associated_Drug_or_Biological_NDC_5                                 0.997671
Physician_License_State_code3                                       0.995469
Product_Category_or_Therapeutic_Area_5                              0.993623
Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_5           0.993483
Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5            0.993420
Covered_or_Noncovered_Indicator_5                                   0.993296
Teaching_Hospital_CCN                                               0.987752
Teaching_Hospital_ID                                                0.987752
Teaching_Hospital_Name                                              0.987752

## Train-Test Split

In [78]:
features = df.drop(columns='Target')
target = df['Target']
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)
model_scores = dict()

**Baseline**

Identifying possible irrelevant columns

In [79]:
columns_to_drop = ['Recipient_Province', 
'Recipient_Postal_Code', 
'Recipient_Primary_Business_Street_Address_Line2',
'Teaching_Hospital_Name', 
'Teaching_Hospital_CCN',
'Teaching_Hospital_ID',
'Physician_Name_Suffix',       
'Program_Year', 
'Physician_Profile_ID', 
'Physician_Last_Name', 
'Physician_First_Name',
'Record_ID',
'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID',
'Physician_Profile_ID',
'Recipient_Zip_Code',
'Date_of_Payment',
'Physician_Middle_Name',
# 'Covered_Recipient_Type', # leaking target info
'Payment_Publication_Date', 
# 'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', # leaking target info
# 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name', # leaking target info
'Unnamed: 0' # Index col from one of the DFs
]

Dropping Columns with any missing value at all

In [80]:
nan_columns = NAs[NAs > 0] 
nan_columns = np.array(nan_columns.index)
to_drop_baseline = list(set(nan_columns) | set(columns_to_drop))

In [81]:
X_train_Baseline = X_train.drop(columns=to_drop_baseline, axis ='columns')

Checking single variable performances to identify leakage issues

In [83]:
objVars = ['Covered_Recipient_Type', # leaking target info
            'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', # leaking target info
            'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name', # leaking target info
           'Form_of_Payment_or_Transfer_of_Value',
           'Dispute_Status_for_Publication', 
           'Delay_in_Publication_Indicator',
           'Related_Product_Indicator',
           'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
           'Change_Type',
           'Total_Amount_of_Payment_USDollars']

single_var = dict()

rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_Baseline, y_train)    

X_train_rus = pd.DataFrame(X_train_rus, columns=X_train_Baseline.columns)

for var in objVars:
    
    if var != 'Total_Amount_of_Payment_USDollars':
        baseline_pipe = Pipeline([
                                ("dummies", OneHotEncoder(handle_unknown='ignore')),
                                ("logreg", LogisticRegression(C=1000000, solver='lbfgs', max_iter=1000))])
    else:
        baseline_pipe = Pipeline([
                                   ("logreg", LogisticRegression(C=1000000, solver='lbfgs', max_iter=1000))])

    # Baseline Training and testing
    logreg = baseline_pipe.fit(X_train_rus[[var]], y_train_rus)
    y_score = logreg.predict_proba(X_train_rus[[var]])
    
    # Store in dict
    single_var[var] = roc_auc_score(y_train_rus, y_score[:, 1])
    
single_var    

KeyboardInterrupt: 

Modifying column to drop list

In [84]:
columns_to_drop += ['Covered_Recipient_Type', # leaking target info
                    'Submitting_Applicable_Manufacturer_or_Applicable_GPO_Name', # leaking target info
                    'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Name' # leaking target info
                    ]

In [85]:
to_drop_baseline = list(set(nan_columns) | set(columns_to_drop))

In [86]:
X_train_Baseline = X_train.drop(columns=to_drop_baseline, axis ='columns')

In [87]:
X_train_Baseline.shape

(1689897, 7)

In [88]:
X_train_Baseline.head()

,Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country,Dispute_Status_for_Publication,Form_of_Payment_or_Transfer_of_Value,Related_Product_Indicator,Total_Amount_of_Payment_USDollars,Change_Type,Delay_in_Publication_Indicator
1633449,United States,No,In-kind items and services,Yes,12.97,UNCHANGED,No
282643,United States,No,In-kind items and services,Yes,15.89,UNCHANGED,No
374929,United States,No,In-kind items and services,Yes,12.32,UNCHANGED,No
547899,United States,No,In-kind items and services,Yes,0.75,UNCHANGED,No
250491,Switzerland,No,Cash or cash equivalent,No,864.86,UNCHANGED,No


## Baselining

Without target encoding

In [89]:
# Defining continuous and categorical variables
objVars = ['Form_of_Payment_or_Transfer_of_Value',
           'Dispute_Status_for_Publication', 
           'Delay_in_Publication_Indicator',
           'Related_Product_Indicator',
           'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country',
           'Change_Type']

contVars = ['Total_Amount_of_Payment_USDollars']

# Random Undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_Baseline, y_train)    
X_train_rus = pd.DataFrame(X_train_rus, columns = X_train_Baseline.columns)
X_train_rus = X_train_rus[objVars + contVars]

In [90]:
# Defining ColumnTransformer
preprocessor = ColumnTransformer(transformers=[("dummies", OneHotEncoder(handle_unknown='ignore'), objVars)],
                                 remainder='passthrough')

# Create pipeplines
baseline_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                          ("logreg", LogisticRegression(C=1000000, solver='lbfgs', max_iter=1000))
                         ])

In [92]:
# Baseline Training and testing
# logreg = baseline_pipe.fit(pd.DataFrame(X_train_rus, columns = X_train.columns), y_train_rus)
# y_score = logreg.predict_proba(pd.DataFrame(X_train_rus, columns = X_train.columns))
# baseline= roc_auc_score(y_train_rus, y_score[:, 1])

baseline = cross_val_score(baseline_pipe, X_train_rus, y_train_rus, scoring='roc_auc', cv=5)
baseline_cv_score = np.mean(baseline)
model_scores['baseline_cv'] = baseline_cv_score

In [93]:
model_scores

{'baseline_cv': 0.9362845959438688}

## Feature engineering ~ Task 3

Imputing NA with 'Missing' values

In [94]:
X_train_engineered = X_train.drop(columns=columns_to_drop)

In [95]:
obj_vars = X_train_engineered.drop(columns=['Total_Amount_of_Payment_USDollars']).columns.values
cont_vars = ['Total_Amount_of_Payment_USDollars']

### Identify high cardinality categorical variables

In [34]:
# Identify which variables to target encode
target_based_encoding = []
for col in obj_vars:
    print(col, len(X_train_engineered[col].unique()))
    
    if len(X_train_engineered[col].unique()) > 100:
        target_based_encoding.append(col)

len(target_based_encoding)

Physician_License_State_code2 55
Product_Category_or_Therapeutic_Area_1 1613
Product_Category_or_Therapeutic_Area_3 399
Product_Category_or_Therapeutic_Area_5 186
Recipient_City 15123
Associated_Drug_or_Biological_NDC_2 537
Covered_or_Noncovered_Indicator_3 3
Recipient_Primary_Business_Street_Address_Line1 299622
Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State 46
Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3 1780
Recipient_State 61
Associated_Drug_or_Biological_NDC_5 86
Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country 33
Covered_or_Noncovered_Indicator_5 3
Dispute_Status_for_Publication 2
Physician_License_State_code3 50
Associated_Drug_or_Biological_NDC_1 1216
Form_of_Payment_or_Transfer_of_Value 6
Related_Product_Indicator 2
Physician_Primary_Type 7
Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4 5
Product_Category_or_Therapeutic_Area_4 297
Product_Category_or_Therapeutic_Area_2 530
Change_Type 3
Associated_Drug_or_Biological_NDC_4 

17

In [35]:
# Final categorical variables
categorical = [cols for cols in obj_vars if cols not in target_based_encoding]
len(categorical) + len(target_based_encoding)

43

In [96]:
# # Train-Test split
# target = X_train_engineered['Target']
# features = X_train_engineered.drop(columns='Target')
# X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=42)

# Random undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_engineered, y_train)
X_train_rus = pd.DataFrame(X_train_rus, columns = X_train_engineered.columns)

Checking for target leakage again since we've added features not used before (for all variables except target encoded ones)

In [97]:
single_var = dict()

cat_pipe = Pipeline([
                        ('Impute', SimpleImputer(strategy='constant', fill_value="Missing")),
                        ("dummies", OneHotEncoder(handle_unknown='ignore')),
                        ("logreg", LogisticRegression(C=1000000, solver='lbfgs', max_iter=1000))])

cont_pipe = Pipeline([
                        ('scalar', StandardScaler()),
                        ("logreg", LogisticRegression(C=1000000, solver='lbfgs', max_iter=1000))])

for var in X_train_rus.columns:
    
    if var != 'Total_Amount_of_Payment_USDollars':
        # Baseline Training and testing
        logreg = cat_pipe.fit(X_train_rus[[var]], y_train_rus)
        y_score = logreg.predict_proba(X_train_rus[[var]])
            
    else:
        # Baseline Training and testing
        logreg = cont_pipe.fit(X_train_rus[[var]], y_train_rus)
        y_score = logreg.predict_proba(X_train_rus[[var]])
    
    # Store in dict
    single_var[var] = roc_auc_score(y_train_rus, y_score[:, 1])
    
single_var    

{'Physician_License_State_code2': 0.5095186927782087,
 'Product_Category_or_Therapeutic_Area_1': 0.8672293096723582,
 'Product_Category_or_Therapeutic_Area_3': 0.5425065143434552,
 'Product_Category_or_Therapeutic_Area_5': 0.5034577969668057,
 'Recipient_City': 0.8290565025076542,
 'Associated_Drug_or_Biological_NDC_2': 0.5961812852369675,
 'Covered_or_Noncovered_Indicator_3': 0.5384249443682315,
 'Recipient_Primary_Business_Street_Address_Line1': 0.9934478274978122,
 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State': 0.6808334293111322,
 'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3': 0.544160652904068,
 'Recipient_State': 0.5971740055670084,
 'Associated_Drug_or_Biological_NDC_5': 0.5011862789434296,
 'Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country': 0.5523730320732043,
 'Covered_or_Noncovered_Indicator_5': 0.5020367967875355,
 'Dispute_Status_for_Publication': 0.5001931503371853,
 'Physician_License_State_code3': 0.5023068004452188,
 'Ass

Adding leakage variables to list of variables to be removed

In [98]:
columns_to_drop += ['Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1', 
                   'Physician_Primary_Type',
                   'Recipient_Primary_Business_Street_Address_Line1',
                   'Physician_Specialty',
                   'Physician_License_State_code1']

In [99]:
X_train_engineered = X_train.drop(columns=columns_to_drop)

In [100]:
obj_vars = X_train_engineered.drop(columns=['Total_Amount_of_Payment_USDollars']).columns.values
cont_vars = ['Total_Amount_of_Payment_USDollars']

In [101]:
# Identify which variables to target encode
target_based_encoding = []
for col in obj_vars:
    print(col, len(X_train_engineered[col].unique()))
    
    if len(X_train_engineered[col].unique()) > 100:
        target_based_encoding.append(col)

len(target_based_encoding)

Physician_License_State_code2 55
Product_Category_or_Therapeutic_Area_1 1613
Product_Category_or_Therapeutic_Area_3 399
Product_Category_or_Therapeutic_Area_5 186
Recipient_City 15123
Associated_Drug_or_Biological_NDC_2 537
Covered_or_Noncovered_Indicator_3 3
Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_State 46
Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3 1780
Recipient_State 61
Associated_Drug_or_Biological_NDC_5 86
Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_Country 33
Covered_or_Noncovered_Indicator_5 3
Dispute_Status_for_Publication 2
Physician_License_State_code3 50
Associated_Drug_or_Biological_NDC_1 1216
Form_of_Payment_or_Transfer_of_Value 6
Related_Product_Indicator 2
Indicate_Drug_or_Biological_or_Device_or_Medical_Supply_4 5
Product_Category_or_Therapeutic_Area_4 297
Product_Category_or_Therapeutic_Area_2 530
Change_Type 3
Associated_Drug_or_Biological_NDC_4 206
Delay_in_Publication_Indicator 1
Name_of_Drug_or_Biological_or_Device_or_Med

14

In [102]:
# Final categorical variables
categorical = [cols for cols in obj_vars if cols not in target_based_encoding]
len(categorical) + len(target_based_encoding)

38

In [103]:
# Random undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_engineered, y_train)
X_train_rus = pd.DataFrame(X_train_rus, columns = X_train_engineered.columns)
X_train_rus = X_train_rus[categorical + cont_vars]

In [59]:
pd.DataFrame(X_train_rus.columns)

,0
0,Physician_License_State_code2
1,Covered_or_Noncovered_Indicator_3
2,Applicable_Manufacturer_or_Applicable_GPO_Maki...
3,Recipient_State
4,Associated_Drug_or_Biological_NDC_5
5,Applicable_Manufacturer_or_Applicable_GPO_Maki...
6,Covered_or_Noncovered_Indicator_5
7,Dispute_Status_for_Publication
8,Physician_License_State_code3
9,Form_of_Payment_or_Transfer_of_Value


### Model without high cardinality categorical variables

In [104]:
# Model without high cardinality categorical variables
# Defining ColumnTransformer
preprocessor = ColumnTransformer(transformers=[("scalar", StandardScaler(), cont_vars),
                                              ("dummies", make_pipeline(SimpleImputer(strategy='constant', fill_value="Missing"),
                                                                        OneHotEncoder(handle_unknown='ignore')), categorical)
                                             ])

# Create pipeplines
take2_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                             ("logreg", LogisticRegression(C=1000000, solver='lbfgs', max_iter=500))
                            ])



In [105]:
# Baseline Training and testing
imputed_model = cross_val_score(take2_pipe, X_train_rus, y_train_rus, scoring='roc_auc', cv=5)
imputed_model_cv_score = np.mean(imputed_model)
model_scores['imputed_model_cv_score'] = imputed_model_cv_score

In [106]:
model_scores

{'baseline_cv': 0.9362845959438688,
 'imputed_model_cv_score': 0.9535514508509628}

### Including Categorical Columns with high cardinality with Target Encoding

Encoding separately as takes a lot of time

In [65]:
# Random undersampling
rus = RandomUnderSampler(random_state=42)
X_train_rus, y_train_rus = rus.fit_resample(X_train_engineered, y_train)
X_train_rus = pd.DataFrame(X_train_rus, columns = X_train_engineered.columns)

In [66]:
# Target Encoding
# # Takes hell lot of time (~10mins)
# # but does the job

# Convert NAs of categorical variables to None
for col in target_based_encoding:
    X_train_rus[col].fillna("None", inplace=True)

# Fitting target encoder
target_enc = TargetEncoder(verbose=1, cols=target_based_encoding, return_df=True, handle_unknown='ignore')
targets_encoded = target_enc.fit_transform(X_train_rus, y_train_rus)


In [107]:
# Defining ColumnTransformer
preprocessor = ColumnTransformer(transformers=[("scalar", StandardScaler(), cont_vars),
                                              ("dummies", make_pipeline(SimpleImputer(strategy='constant', fill_value="Missing"),
                                                                        OneHotEncoder(handle_unknown='ignore')), categorical)
                                             ], remainder='passthrough')

# Create pipeplines
take3_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                             ("logreg", LogisticRegression(C=1000000, solver='lbfgs', max_iter=500))
                            ])



In [108]:
# Baseline Training and testing
target_enc_model = cross_val_score(take3_pipe, targets_encoded, y_train_rus, scoring='roc_auc', cv=5)
target_enc_model_cv_score = np.mean(target_enc_model)
model_scores['target_enc_model'] = target_enc_model_cv_score


In [109]:
model_scores

{'baseline_cv': 0.9362845959438688,
 'imputed_model_cv_score': 0.9535514508509628,
 'target_enc_model': 0.9829174974299428}

## Trying linear SVC

In [112]:
# Defining ColumnTransformer
preprocessor = ColumnTransformer(transformers=[("scalar", StandardScaler(), cont_vars),
                                              ("dummies", make_pipeline(SimpleImputer(strategy='constant', fill_value="Missing"),
                                                                        OneHotEncoder(handle_unknown='ignore')), categorical)
                                             ], remainder='passthrough')

# Create pipeplines
svc_pipe = Pipeline(steps=[('preprocessor', preprocessor),
                                 ("SVC", LinearSVC())
                                ])



In [115]:
# Baseline Training and testing
svc_model = cross_val_score(svc_pipe, targets_encoded, y_train_rus, scoring='roc_auc', cv=5)
svc_model_cv_score = np.mean(svc_model)
model_scores['svc_model'] = svc_model_cv_score


In [116]:
model_scores

{'baseline_cv': 0.9362845959438688,
 'imputed_model_cv_score': 0.9535514508509628,
 'target_enc_model': 0.9829174974299428,
 'svc_model': 0.9811154450064498}

## Code for drawing ROC curve

In [ ]:
y_score = logreg.fit(pd.DataFrame(X_train_rus, columns = X_train.columns), 
                     pd.DataFrame(y_train_rus)).predict_proba(X_test)

In [ ]:
preds = logreg.predict(X_test)
tn, fp, fn, tp  = confusion_matrix(y_test, preds).ravel()
print([tn, fp])
print([fn, tp])

In [ ]:
roc_auc_score(y_test, y_score[:, 1])

In [ ]:
plot_roc(y_test, list(y_score[:, 1]))

In [ ]:
average_precision_score(y_test, y_score)

In [ ]:
def plot_roc(y_test, y_score):
    
    fpr, tpr, thresholds = roc_curve(y_test, y_score)
    
    roc_auc = auc(fpr, tpr)
    
    lw = 2
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
y_prob = logreg.predict_proba(X_test)